# Question 2
### Working with ADQL

In [2]:
# Basic imports
import numpy as np
import matplotlib.pyplot as plt

from astropy.table import Table

In [4]:
# Download the GAIA Data
from astroquery.gaia import Gaia

From the Gaia/2MASS Tables, we need:
- Gaia BP-RP
- Gaia absolute g
- 2Mass J-Ks
- 2Mass Apparent H

In [ ]:
# Select relevant columns and join with the 2Mass table. Constrain ra/dec to
# a 1 degree circle from Ruprecht 147
query = """
SELECT gaia.source_id, gaia.ra, gaia.dec, gaia.parallax, gaia.phot_g_mean_mag, 
gaia.bp_rp, tmass.j_m, tmass.ks_m, tmass.h_m
FROM gaiadr3.gaia_source AS gaia
JOIN gaiadr3.tmass_psc_xsc_best_neighbour AS xmatch USING (source_id)
JOIN gaiadr3.tmass_psc_xsc_join AS xjoin USING (clean_tmass_psc_xsc_oid)
JOIN gaiadr1.tmass_original_valid AS tmass ON
   xjoin.original_psc_source_id = tmass.designation
WHERE phot_g_mean_mag < 14
   AND CONTAINS(
     POINT('ICRS', gaia.ra, gaia.dec),
     CIRCLE('ICRS', 289.074, -16.323, 1.0)
   ) = 1;
"""
job = Gaia.launch_job_async(query)
result = job.get_results()
df = result.to_pandas()
display(df.head())
print("Dataframe has shape", df.shape)


In [ ]:
# Manipulate column names to be easier to work with, and add absolute g magnitude
df.columns = ["ID", "RA", "DEC", "parallax", "g_m", "bp_rp", "j_m", "ks_m", "h_m"]
df["g_M"] = df["g_m"]+5*np.log10(df["parallax"]/1000)+5

ValueError: Length mismatch: Expected axis has 10 elements, new values have 9 elements